In [38]:
import city_scraper

import time
import pickle as pkl
import datetime
import csv
# Scraping items
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from IPython.core.display import display, HTML
import concurrent.futures
from collections import Counter
import numpy as np
from scipy import stats
import pandas as pd
from subprocess import call
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


cwd=os.getcwd()
approot=cwd.strip('preprocessing')

In [180]:
chromedriver="/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [27]:
urls=pkl.load(open(approot+'good_data/Los_Angeles/urls','rb'))

In [29]:
loc_info=pkl.load(open(approot+'good_data/Los_Angeles/loc_info.pkl','rb'))

In [35]:
urls[:2]

['https://www.tripadvisor.com/Attraction_Review-g32655-d147966-Reviews-The_Getty_Center-Los_Angeles_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32655-d116887-Reviews-Griffith_Observatory-Los_Angeles_California.html']

In [51]:
def make_loc_info(urls_final):
    location_info2={}
    ct=0
    for url in urls_final:
        print(f"Process is {round(100*ct/len(urls_final),3)} % through",end="\r")
        ct+=1
        driver.get(url)
        soup=BeautifulSoup(driver.page_source,'html.parser')
        try:
            rating=float(soup.find('span',class_='overallRating').text)
        except:
            rating='None'
        try:
            tags=[x.text for x in soup.find('div',class_='detail').find_all('a',href=True)]
        except:
            tags=['None']
        name=soup.find(id='HEADING').text
        location_info2[name]=[rating,tags]
        #Should be inserting into sql.
    pkl.dump(location_info2,open(approot+'good_data/Los_Angeles/loc_info.pkl','wb'))

    return location_info2

In [52]:
loc_info=make_loc_info(urls)

In [53]:
loc_info

{'The Getty Center': [5.0, ['Museums', 'Specialty Museums']],
 'Griffith Observatory': [4.5, ['Museums', 'Observatories & Planetariums']],
 'Universal Studios Hollywood': [4.5,
  ['Water & Amusement Parks', 'Theme Parks']],
 '\nSightseeing Tours in Los Angeles\u200e\n': ['None',
  ['We Love LA Tour', '723 Reviews ']],
 'Diorama-museum of Bhagavad-gita': [5.0,
  ['Museums', 'Art Museums', 'Specialty Museums']],
 '\nCity Tours in Los Angeles\u200e\n': ['None',
  ['We Love LA Tour', '724 Reviews ']],
 '\nPrivate Tours in Los Angeles\u200e\n': ['None',
  ['We Love LA Tour', '723 Reviews ']],
 'The Nethercutt Collection': [5.0, ['Museums', 'Specialty Museums']],
 '\nWalking Tours in Los Angeles\u200e\n': ['None',
  ['Tourific Escapes', '800 Reviews ']],
 '\nRoom Escape Games in Los Angeles\u200e\n': ['None', ['None']],
 'Abbot Kinney Boulevard': [4.5,
  ['Sights & Landmarks', 'Points of Interest & Landmarks']],
 'Lake Hollywood Park': [4.5, ['Nature & Parks']],
 'Venice Canals Walkway': [4.

In [41]:
url=urls[0]
driver.get(url)
soup=BeautifulSoup(driver.page_source,'html.parser')

In [49]:
float(soup.find('span',class_='overallRating').text)

5.0

In [67]:
user_ids3=list(filter(lambda x: len(x.split(' '))==1 and len(x)!=0,user_ids2))

In [ ]:
'https://www.tripadvisor.com/members/DianneKamerling'

In [69]:
user_urls=['https://www.tripadvisor.com/members/'+x for x in user_ids3]

In [13]:
def make_attraction_reviews(urls):
    tail='-'+urls[1].split('-')[-1]
    attraction_reviews={}
    ct=0
    for url in urls:
        print(ct)
        ct+=1
        try:
            driver.get(url)
            soup=BeautifulSoup(driver.page_source,'html.parser')
            url_list=url.split('-')
            name='-'.join(url_list[-2:])
            template='-'.join(url_list[:-2])+'-or{}-'+name
            last_num=soup.find(class_="pageNum last taLnk ")
            last_page=int(last_num['data-page-number'])
            review_urls=[template.format('')]
            for i in range(1,last_page):
                review_urls.append(template.format(i*10))
            attraction_reviews[name.strip(tail)]=review_urls
        except:
            pass
    return attraction_reviews

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
# pkl.dump(attraction_reviews,open(approot+'good_data/Los_Angeles/attraction_reviews_urls.pkl','wb'))
attraction_reviews=pkl.load(open(approot+'good_data/Los_Angeles/attraction_reviews_urls.pkl','rb'))


In [7]:
def make_user_ids(attraction_reviews):
    user_ids=[]
    keys=list(attraction_reviews.keys())
    L=len(keys)
    ct=0
    for k in keys:
        ct+=1
        print(f"Process is {round(100*ct/L,3)} % through",end="\r")
        urls=attraction_reviews[k]
        for url in urls:
            try:
                driver.get(url)
                soup=BeautifulSoup(driver.page_source,'html.parser')
                ids=soup.find_all(class_="info_text")
                for ID in ids:
                    t1=ID.text
                    t2=ID.find('strong').text
                    t3=t1.strip(t2)
                    user_ids.append(t3)
            except:
                pass
    return user_ids
        

In [186]:
len(user_ids)

40941

In [10]:
user_ids2=list(set(user_ids))
user_ids3=list(filter(lambda x: len(x.split(' '))==1 and len(x)!=0,user_ids2))
user_urls=['https://www.tripadvisor.com/members/'+x for x in user_ids3]

In [12]:
pkl.dump(user_ids2,open(approot+'good_data/Los_Angeles/user_ids.pkl','wb'))

In [184]:
def scrape_reviews(urls):
    count=0
    tmp={}
    users={}
    ct=0
    for url in urls:
        print(f"Process is {round(100*ct/len(urls),3)} % through",end="\r")
        ct+=1
        name=url[36:]
        driver.get(url)
        soup=BeautifulSoup(driver.page_source,'html.parser')
#         pages=int(soup.find_all(class_="cs-paginate-goto")[-1].text)
        reviews=soup.find_all('li',class_='cs-review')
        ranks={}
        for review in reviews:

            try:
                attraction_name=review.find_all(class_='cs-review-location')[0].text
                rating=int([str(tag) for tag in review.find_all()][-3][-10:-9])
            except:
                pass
            try:
                points=int(soup.find(class_="points").text.strip(' ').replace(',',''))
            except:
                points='None'
            try:
                level=int(soup.find(class_="level tripcollectiveinfo").text.split(' ')[1])
            except:
                level='None'
            try:
                readers=int(soup.find(class_="currentBadge").find(class_="badgeSubtext").text.split(' ')[1].replace(',',''))
            except:
                readers='None'
            try:
                stats={'points':points,'level':level,'readers':readers}
            except:
                stats={}

            if 'Los Angeles:' == attraction_name[:12]:
                ranks[attraction_name]=rating
                
        if len(ranks)!=0:
            users[name]={'ratings':ranks,'stats':stats}
            tmp[name]={'ratings':ranks,'stats':stats}
        count+=1
        if count%100==0:
            
            with open(approot+f'good_data/Los_Angeles/file_{4400+count}.pkl','wb') as f:
                pkl.dump(tmp,f)
#             print(count)
            tmp={}
    
    return users
        

In [183]:
.54559*len(user_urls)

11020.918

In [185]:
users=scrape_reviews(user_urls[11000:])

In [122]:
test='https://www.tripadvisor.com/members/HavePlanetWillTravel'
driver.get(test)
soup=BeautifulSoup(driver.page_source,'html.parser')